In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

import pickle

from nltk.tokenize import sent_tokenize
import pandas as pd

from glob import glob 
import datetime

In [2]:
#del BERTDataset

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
 
    def __len__(self):
        return (len(self.labels))

## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [3]:
keyword_dict={
    "보험" : ["삼성생명", "생명보험", "생보사", "금융위원회", "실손보험", "손해보험", "손보사", "변액보험", "연금보험", " 종신보험", "금융+보험", "약관대출", "금융당국+보험", "금감원+보험", "금융감독원+보험", "GA+보험", "보험업계", "보험+디지털", "보험+핀테크"],
    "디지털" : ["빅데이터", "AI -조류", "딥러닝", "인공지능", "핀테크", "테크핀", "빅데이터+보험", "AI+보험", "인공지능+보험", "삼성생명+디지털"],
}

In [4]:
from get_news_data import get_news_data

keyword_group="디지털"

news_dic = get_news_data(keyword_group,keyword_dict[keyword_group])

    
dataset =[]
index_to_cont = {}
for item in news_dic.items():
    
    for idx, cont in enumerate(item[1]["content"]):
        
        cont_index = item[0] + str(idx).zfill(3)
                
        dataset.append([cont, cont_index])
        index_to_cont[cont_index]={"cont":cont}


_, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dataset = BERTDataset(dataset, 0, 1, tok, max_len, True, False)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=5, shuffle=False)

  0%|          | 0/10 [00:00<?, ?it/s]

query list :  ['빅데이터', 'AI -조류', '딥러닝', '인공지능', '핀테크', '테크핀', '빅데이터+보험', 'AI+보험', '인공지능+보험', '삼성생명+디지털']


100%|██████████| 10/10 [00:18<00:00,  1.81s/it]


using cached model
using cached model
using cached model


In [5]:
dataset =[]
index_to_cont = {}
for item in news_dic.items():
    
    for idx, cont in enumerate(item[1]["content"]):
        
        cont_index = item[0] + str(idx).zfill(3)
                
        dataset.append([cont, cont_index])
        index_to_cont[cont_index]={"cont":cont}

In [6]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifiernn.Linear(hidden_size , num_classes)
        
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        if self.dr_rate:
            out = self.dropout(pooler)

        return self.classifier(out)

In [7]:
model_name = "./model/[SOTA]kobert_downstream_3epoch.sav"
model = pickle.load(open(model_name, 'rb'))

##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
result_dict = {}

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    #label = label.long().to(device)
    #order_ratios=torch.from_numpy(np.asarray(np.float32(order_ratios))).long().to(device)
    score = model(token_ids, valid_length, segment_ids)
    score = score.to("cpu")
    index_to_cont[label[0]]["score"]=score.detach().numpy()[0][1]

100%|██████████| 1264/1264 [00:14<00:00, 86.25it/s]


In [9]:
for item in tqdm(news_dic.items()):
    news_index = item[0]
    cont_index = [x[0] for x in index_to_cont.items() if x[0][:10]==news_index]
    output_sorted = sorted({k: index_to_cont[k] for k in index_to_cont.keys() if k in cont_index}.items(), key=lambda x: x[1]["score"], reverse=True)
    
    news_dic[item[0]]["ext_summary"] = [int(x[0][-3:]) for x in output_sorted][:3]
    


100%|██████████| 57/57 [00:00<00:00, 1350.62it/s]


In [10]:
keyword_list = []
title_list =[]
summary_list = []
content_list =[]
url_list=[]
for item in news_dic.items():
    cont = news_dic[item[0]]["content"]
    ext_index = news_dic[item[0]]["ext_summary"]

    
    print("query : ", item[1]["query"])
    print("title : ", item[1]["title"])
    print("summary : \n", "\n".join([cont[x] for x in ext_index]))
    print("content : \n", ".".join(item[1]["content"]))
    print("url : ", item[1]["url"])
    print("="*50)
    

query :  빅데이터
title :  LGU+, 세종시 자율주행 빅데이터 관제센터 구축
summary : 
 		   LG유플러스는 세종특별자치시의 자율주행 빅데이터 관제센터 및 플랫폼 구축 사업자로 LG유플러스 컨소시엄이 선정돼 관제센터 구축 및 운영을 맡는다고 31일 밝혔다
   특히 이번 사업에서 LG유플러스는 세종시 내에서 진행되는 자율주행 실증 환경의 다양화를 위해 기존 실증구간을 60km 범위까지 늘릴 수 있도록 정밀지도를 확대 적용할 계획이다
 LG유플러스 제공 세종시 자율주행 빅데이터 관제센터 구축 및 운영 사업은 자율주행 실증 규제자유특구로 지정된 세종시에서 자율주행 실증 차량의 데이터를 수집해 분석하고 관제 컨트롤 타워 구축 및 향후 사업화를 목표로 하고 있다
content : 
 		   LG유플러스는 세종특별자치시의 자율주행 빅데이터 관제센터 및 플랫폼 구축 사업자로 LG유플러스 컨소시엄이 선정돼 관제센터 구축 및 운영을 맡는다고 31일 밝혔다.    LG유플러스 컨소시엄이 구축하는 세종시 자율주행 빅데이터 관제센터 내의 관제실 및 상황판 예상도. LG유플러스 제공 세종시 자율주행 빅데이터 관제센터 구축 및 운영 사업은 자율주행 실증 규제자유특구로 지정된 세종시에서 자율주행 실증 차량의 데이터를 수집해 분석하고 관제 컨트롤 타워 구축 및 향후 사업화를 목표로 하고 있다.   LG유플러스는 이를 위해 빅데이터 기반의 관제센터를 구축한다. 자율주행 빅데이터 관제센터는 자율주행 실증차량의 운행·인프라·교통 정보 데이터를 수집해 분석한 뒤 이를 활용할 수 있도록 공유하는 빅데이터 플랫폼을 바탕으로 하며 5G도 시범적으로 적용한다.   관제센터에서는 세종시 자율주행 실증에 투입하는 차량에 대한 △실시간 관제 및 현황 관리 △실증 테스트 관리 △V2X 인프라 관리 △정밀지도 구축 및 관리 등 통합 관제가 가능하다.   특히 이번 사업에서 LG유플러스는 세종시 내에서 진행되는 자율주행 실증 환경의 다양화를 위해 기존 실증구간을 60km 범위까지 늘

In [11]:
keyword_list = []
title_list =[]
summary_list = []
content_list =[]
url_list=[]
press_list=[]
for item in news_dic.items():
    cont = news_dic[item[0]]["content"]
    ext_index = news_dic[item[0]]["ext_summary"]
    
    """
    print("query : ", item[1]["query"])
    print("title : ", item[1]["title"])
    print("summary : \n", ".".join([cont[x] for x in ext_index]))
    print("content : \n", ".".join(item[1]["content"]))l
    print("="*50)
    """
    
    keyword_list.append(item[1]["query"])
    title_list.append(item[1]["title"])
    summary_list.append("\n".join([cont[x] for x in ext_index]))
    content_list.append("\n".join(item[1]["content"]))
    url_list.append(item[1]["url"])
    press_list.append(item[1]["press"])
    
today = datetime.date.today()
today.isoformat()


result=pd.DataFrame()

result["keyword"] = keyword_list
result["title"] = title_list
result["summary"] = summary_list
result["content"] = content_list
result["url"] = url_list
result["date"] = today
result["press"]=press_list


fname_list = glob("./result/*")

saved_url=[]
for fname in fname_list:
    tmp = pd.read_csv(fname, sep="\t")
    saved_url += list(tmp["url"])
    
result = result[~result["url"].isin(saved_url)]

print(len(result))
result.to_csv("./"+today.isoformat()+"_"+keyword_group+".tsv", sep="\t", encoding="utf8", index=False)

51


In [12]:
import smtplib
from email.mime.text import MIMEText

result_text="이 메시지는 NAVER에서 뉴스를 크롤링한 후 BERT를 사용하여 추출요약한 결과입니다\n"

for title, summary, url, press in zip(result["title"], result["summary"], result["url"], result["press"]):
    result_text += "\n"
    result_text += "-"*200
    result_text += "\n\n"
    result_text += "<"+title.strip()+"> : "+ press
    result_text += "\n"*2
    result_text += summary.strip()
    result_text += "\n"*2
    result_text += url
    result_text += "\n"
    
smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.ehlo()      # say Hello
smtp.starttls()  # TLS 사용시 필요
smtp.login('godjsw1@gmail.com', '2wldmsdl1')

today = datetime.date.today()
Subject = '[뉴스요약][{keyword}]'.format(keyword=keyword_group) + today.isoformat().split("-")[0] +"년 " +today.isoformat().split("-")[1] +"월 " +today.isoformat().split("-")[2] +"일"

msg = MIMEText(result_text)
msg['Subject'] = Subject
msg['To'] = 'sangwon.joo@samsung.com'
smtp.sendmail('godjsw1@gmail.com', 'sangwon.joo@samsung.com', msg.as_string())
#smtp.sendmail('godjsw1@gmail.com', 'sangwon.joo@samsung.com', msg.as_string())
#smtp.sendmail('godjsw1@gmail.com', 'peter8.lee@samsung.com', msg.as_string())
#smtp.sendmail('godjsw1@gmail.com', 'rami.kim@samsung.com', msg.as_string())
#smtp.sendmail('godjsw1@gmail.com', 'jy927.nam@samsung.com', msg.as_string())
#smtp.sendmail('godjsw1@gmail.com', 'g0835.min@samsung.com', msg.as_string())
 
smtp.quit()


(221, b'2.0.0 closing connection v26sm15617157pff.195 - gsmtp')